<br>
<u>Notebook Six</u> | 
<a href=https://leone.gdn/DLL target=_blank>Report</a> | 
<a href=https://github.com/andrealeone/DLL>Repository</a>
<br><br>
<b>Task 1: Attribute classifiers</b><br><br>
Andrea Leone, Alessandro E. Piotti<br>
Deep Learning Lab — University of Trento<br>
December 2021
<hr><br><br>

In [1]:
import project 

import os
import pickle
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import random
import collections

import torch
import torch.nn          as nn
import torch.optim       as optim
import torchvision
import sklearn

from tqdm.notebook   import tqdm

project.notebook()

In [2]:
images, annotations = project.load_resources() 

<hr><br><br>

## `Colors_Up` classifier

<br><br>Load the train set, isolate the attribute as label, crop the image, and compute class weights

In [3]:
attribute  = 'colors_up' 

irm        = torchvision.io.ImageReadMode.RGB
split_val  = 640
dictionary = dict()
colors     = [
    'upblack', 'upwhite', 'upred', 'uppurple',
    'upyellow', 'upgray', 'upblue', 'upgreen'
]

for image_file in tqdm(images):
    
    image_id  = int(image_file.split('/')[-1].split('_')[0])
    array     = [(annotations.loc[image_id][color] - 1) for color in colors]
    tensor    = torchvision.io.read_image( image_file,irm ).float()
    
    if 1 in array:
        label = (array.index(1) + 1)
    else:
        label = 0
    
    ct = list()
    for c,channel in enumerate(tensor):
        ct.append( tensor[c][20:80].numpy() )
    
    tensor    = torch.tensor(ct).float()
    
    channels, height, width = tensor.shape
    tensor    = tensor.reshape(1, channels, height, width)
    
    if image_id not in dictionary:
        dictionary[image_id] = list()
        dictionary[image_id].append([tensor, label])
    else:
        dictionary[image_id].append([tensor, label])
    
data = list(dictionary.items())
random.Random(3).shuffle(data)

train_set, test_set = project.split_dataset(data,split_val) 

  0%|          | 0/12989 [00:00<?, ?it/s]

<hr><br><br>Fit a pretrained ResNet50 architecture

In [ ]:
v = '3_2'                       # model version 
e =  0                          # starting epoch
f =  0                          # feature index 


model     = torchvision.models.resnet50(pretrained=True) 
model.fc  = nn.Linear(
    in_features=model.fc.in_features,
    out_features=2, bias=True
)

if e > 0:
    w = './models/{}_resnet50.v{}.x{}.net'.format(attribute, v, e)
    model.load_state_dict( torch.load(w) )


optimizer = optim.AdamW(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


for i in range(10): 

    model_name = '{}_resnet50.v{}.x{}.net'.format(attribute, v, e+i+1)

    p = project.train(model, train_set,
        criterion, optimizer, epochs=1, li=500, monobatch=True
    )

    r = project.test(model, test_set, monobatch=True)

    project.save(model, './models/{}'.format(model_name))

In [ ]:
# SCORE BOARD 
# 
# net_design_name    | epochs | accuracy           | precision | recall
#                    |        |                    |           |
# resnet50 v3.2 f0   |  1     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f0   |  2     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f0   |  3     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f0   |  4     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f0   |  5     | 0.5                | 0.5       | 0.5
# resnet50 v3.2 f0   |  6     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f0   |  7     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f0   |  8     | 0.4962             | 0.4941309 | 0.4962963
# resnet50 v3.2 f0   |  9     | 0.5296             | 0.5612523 | 0.5296296
# resnet50 v3.2 f0   | 10     | 0.3592             | 0.3564151 | 0.3592593
# resnet50 v3.2 f0   | 11     | 0.5408             | 0.5442465 | 0.5407407
# resnet50 v3.2 f0   | 12     | 0.44               | 0.4222350 | 0.44
# resnet50 v3.2 f0   | 13     | 0.4593             | 0.4388436 | 0.4592593
# resnet50 v3.2 f0   | 14     | 0.44               | 0.4222350 | 0.44
# resnet50 v3.2 f0   | 15     | 0.52               | 0.5193207 | 0.52
# resnet50 v3.2 f0   | 16     | 0.49               | 0.4643549 | 0.49
# resnet50 v3.2 f0   | 17     | 0.4925             | 0.4884259 | 0.4925926
# resnet50 v3.2 f0   | 18     | 0.5481             | 0.5497195 | 0.5481481
# resnet50 v3.2 f0   | 19     | 0.5370             | 0.5379555 | 0.5370370
# resnet50 v3.2 f0   | 20     | 0.5529             | 0.5551471 | 0.5518519
# resnet50 v3.2 f0   | 21     | 0.5148             | 0.5262034 | 0.5148148
# resnet50 v3.2 f0   | 22     | 0.5629             | 0.5712335 | 0.5629630
# resnet50 v3.2 f0   | 23     | 0.5259             | 0.5548080 | 0.5259259
# resnet50 v3.2 f0   | 24     | 0.52               | 0.6099946 | 0.52
#                    |        |                    |           |
# resnet50 v3.2 f1   |  1     | 0.8022388059701493 | 0.8426136 | 0.8022388
# resnet50 v3.2 f1   |  2     | 0.7910447761194029 | 0.8381211 | 0.7910448
# resnet50 v3.2 f1   |  3     | 0.8619402985074627 | 0.8784649 | 0.8619403
# resnet50 v3.2 f1   |  4     | 0.830223880597015  | 0.8691230 | 0.8302239
# resnet50 v3.2 f1   |  5     | 0.8656716417910447 | 0.8777906 | 0.8656716
# resnet50 v3.2 f1   |  6     | 0.8115671641791045 | 0.8116757 | 0.8115672
# resnet50 v3.2 f1   |  7     | 0.83955223880597   | 0.8465189 | 0.8395522
# resnet50 v3.2 f1   |  8     | 0.8861940298507462 | 0.8936981 | 0.8861940
#                    |        |                    |           |
# resnet50 v3.2 f2   |  1     | 0.604165           | 0.6359810 | 0.6041667
# resnet50 v3.2 f2   |  2     | 0.7762681159420289 | 0.7792457 | 0.7762681
# resnet50 v3.2 f2   |  3     | 0.842391304347826  | 0.8582102 | 0.8423913
# resnet50 v3.2 f2   |  4     | 0.5905797101449275 | 0.6755189 | 0.5905797
# resnet50 v3.2 f2   |  5     | 0.730072463768116  | 0.7581339 | 0.7300725
# resnet50 v3.2 f2   |  6     | 0.6992753623188406 | 0.7165725 | 0.6992754
# resnet50 v3.2 f2   |  7     | 0.6820652173913043 | 0.7314596 | 0.6820652
# resnet50 v3.2 f2   |  8     | 0.8079710144927537 | 0.8139217 | 0.8079710
# resnet50 v3.2 f2   |  9     | 0.677536231884058  | 0.6779310 | 0.6775362
#                    |        |                    |           |
# resnet50 v3.2 f3   |  1     | 0.7317073170731707 | 0.7408782 | 0.7317073
# resnet50 v3.2 f3   |  2     | 0.5121951219512195 | 0.7530864 | 0.5121951
# resnet50 v3.2 f3   |  3     | 0.8130081300813008 | 0.8500259 | 0.8130081
# resnet50 v3.2 f3   |  4     | 0.6138211382113821 | 0.6147388 | 0.6138211
# resnet50 v3.2 f3   |  5     | 0.6544715447154471 | 0.6611280 | 0.6544715
# resnet50 v3.2 f3   |  6     | 0.6138211382113821 | 0.6937444 | 0.6138211
# resnet50 v3.2 f3   |  7     | 0.7439024390243902 | 0.7452153 | 0.7439024
# resnet50 v3.2 f3   |  8     | 0.7723577235772358 | 0.7783181 | 0.7723577
# resnet50 v3.2 f3   |  9     | 0.7560975609756098 | 0.7561653 | 0.7560976
#                    |        |                    |           |
# resnet50 v3.2 f4   |  1     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f4   |  2     | 0.77               | 0.778     | 0.77
# resnet50 v3.2 f4   |  3     | 0.9                | 0.9167    | 0.9
# resnet50 v3.2 f4   |  4     | 0.64               | 0.7083333 | 0.64
# resnet50 v3.2 f4   |  5     | 0.97               | 0.96875   | 0.97
# resnet50 v3.2 f4   |  6     | 0.9                | 0.9166667 | 0.9
# resnet50 v3.2 f4   |  7     | 0.97               | 0.96875   | 0.97
# resnet50 v3.2 f4   |  8     | 0.94               | 0.9411765 | 0.94
# resnet50 v3.2 f4   |  9     | 0.94               | 0.9411765 | 0.94
#
# resnet50 v3.2 f5   |  1     | 0.5030487804878051 | 0.7507645 | 0.5030488
# resnet50 v3.2 f5   |  2     | 0.8841463414634152 | 0.9013986 | 0.8841463
# resnet50 v3.2 f5   |  3     | 0.8932926829268294 | 0.8940105 | 0.8932927
# resnet50 v3.2 f5   |  4     | 0.9207317073170734 | 0.9270833 | 0.9207317
# resnet50 v3.2 f5   |  5     | 0.8353658536585372 | 0.8697327 | 0.8353659
# resnet50 v3.2 f5   |  6     | 0.8018292682926833 | 0.8078849 | 0.8018293
# resnet50 v3.2 f5   |  7     | 0.7865853658536591 | 0.7951900 | 0.7865854
# resnet50 v3.2 f5   |  8     | 0.7804878048780495 | 0.7825468 | 0.7804878
# resnet50 v3.2 f5   |  9     | 0.8567073170731713 | 0.8583193 | 0.8567073
# resnet50 v3.2 f5   | 10     | 0.8201219512195121 | 0.8641054 | 0.8201220
# resnet50 v3.2 f5   | 11     | 0.875              | 0.8756844 | 0.875
# resnet50 v3.2 f5   | 12     | 0.8963414634146342 | 0.9100631 | 0.8963415
#
# resnet50 v3.2 f6   |  1     | 0.5274509803921569 | 0.5381410 | 0.5274510
# resnet50 v3.2 f6   |  2     | 0.5137254901960784 | 0.5410137 | 0.5137255
# resnet50 v3.2 f6   |  3     | 0.4980392156862745 | 0.4161736 | 0.4980392
# resnet50 v3.2 f6   |  4     | 0.5039215686274510 | 0.5039730 | 0.5039216
# resnet50 v3.2 f6   |  5     | 0.4862745098039215 | 0.3618635 | 0.4862745
# resnet50 v3.2 f6   |  6     | 0.5294117647058824 | 0.5302319 | 0.5294118
# resnet50 v3.2 f6   |  7     | 0.5                | 0.5       | 0.5
# resnet50 v3.2 f6   |  8     | 0.5                | 0.5       | 0.5
# resnet50 v3.2 f6   |  9     | 0.5352941176470588 | 0.5353207 | 0.5352941
# resnet50 v3.2 f6   | 10     | 0.5588235294117647 | 0.5619172 | 0.5588235
# resnet50 v3.2 f6   | 11     | 0.5607843137254902 | 0.6115234 | 0.5607843
# resnet50 v3.2 f6   | 12     | 0.5843137254901961 | 0.6139221 | 0.5843137
#                    |        |                    |           |
# resnet50 v3.2 f7   |  1     | 0.369319           | 0.3679368 | 0.3693182
# resnet50 v3.2 f7   |  2     | 0.369319           | 0.2319205 | 0.3693182
# resnet50 v3.2 f7   |  3     | 0.3125             | 0.2282426 | 0.3125
# resnet50 v3.2 f7   |  4     | 0.482955           | 0.3456140 | 0.4829545
# resnet50 v3.2 f7   |  5     | 0.460228           | 0.4423545 | 0.4602273
# resnet50 v3.2 f7   |  6     | 0.494319           | 0.4152216 | 0.4943182
# resnet50 v3.2 f7   |  7     | 0.676137           | 0.8034483 | 0.6761364
# resnet50 v3.2 f7   |  8     | 0.8634             | 0.8705263 | 0.8636364
# resnet50 v3.2 f7   |  9     | 0.857955           | 0.8591139 | 0.8579545
# resnet50 v3.2 f7   | 10     | 0.80682            | 0.8202156 | 0.8068182
#                    |        |                    |           |
# resnet50 v3.2 f8   |  1     | 0.6529680365296804 | 0.6532876 | 0.6529680
# resnet50 v3.2 f8   |  2     | 0.5091324200913242 | 0.7523041 | 0.5091324
# resnet50 v3.2 f8   |  3     | 0.5                | 0.5       | 0.5
# resnet50 v3.2 f8   |  4     | 0.8584474885844748 | 0.8591964 | 0.8584475
# resnet50 v3.2 f8   |  5     | 0.4977168949771689 | 0.4160920 | 0.4977169
# resnet50 v3.2 f8   |  6     | 0.7123287671232876 | 0.7295597 | 0.7123288
# resnet50 v3.2 f8   |  7     | 0.54337899543379   | 0.6278184 | 0.5433790
# resnet50 v3.2 f8   |  8     | 0.4977168949771689 | 0.2494279 | 0.4977169
# 

In [ ]:
# PREVIOUS SCORE BOARD 
# 
# net_design_name | epochs | accuracy           | precision | recall
#                 |        |                    |           |
# resnet50 v2 f1  |  1     | 0.8115671641791045 | 0.8131410 | 0.8115671
# resnet50 v2 f1  |  2     | 0.8302238805970149 | 0.8551502 | 0.8302238
# resnet50 v4 f1  |  2     | 0.8488805970149254 | 0.8490020 | 0.8488805
# resnet50 v2 f1  |  3     | 0.8488805970149254 | 0.8542517 | 0.8488805
# resnet50 v3 f1  |  3     | 0.8712686567164180 | 0.8735623 | 0.8712686
# resnet50 v4 f1  |  3     | 0.8320895522388060 | 0.8396559 | 0.8320895
# resnet50 v2 f1  |  4     | 0.7481343283582089 | 0.7837943 | 0.7481343
# resnet50 v3 f1  |  4     | 0.8694029850746269 | 0.8695882 | 0.8694029
# resnet50 v3 f1  |  5     | 0.8582089552238806 | 0.8771106 | 0.8582089
# resnet50 v3 f1  |  6     | 0.8861940298507462 | 0.8868457 | 0.8861940
# resnet50 v3 f1  |  7     | 0.8675373134328358 | 0.8836974 | 0.8675373
#
# resnet50 v3 f2  |  2     | 0.4646739130434782 | 0.4337905 | 0.4646739
# resnet50 v3 f2  |  3     | 0.5036231884057971 | 0.5053893 | 0.5036231
# resnet50 v3 f2  |  4     | 0.6286231884057971 | 0.7335581 | 0.6286231
# resnet50 v3 f2  |  5     | 0.5878623188405797 | 0.6336094 | 0.5878623
# resnet50 v3 f2  |  6     | 0.6268115942028986 | 0.6337820 | 0.6268115
# resnet50 v3 f2  |  7     | 0.6195652173913043 | 0.6199194 | 0.6195652
#
# resnet50 v3 f3  |  1     | 0.7560975609756098 | 0.8293243 | 0.7560976
# resnet50 v3 f3  |  2     | 0.6666666666666666 | 0.7612141 | 0.6666667
# resnet50 v3 f3  |  3     | 0.8699186991869918 | 0.8699187 | 0.8699187
# resnet50 v3 f3  |  4     | 0.8414634146341463 | 0.8715478 | 0.8414634
# resnet50 v3 f3  |  5     | 0.7520325203252033 | 0.7548797 | 0.7520325
# resnet50 v3 f3  |  6     | 0.7154471544715447 | 0.7182750 | 0.7154472
#
# resnet50 v3 f4  |  1     | 0.4333333333333333 | 0.4147727 | 0.4333333
# resnet50 v3 f4  |  2     | 0.5                | 0.5       | 0.5
# resnet50 v3 f4  |  3     | 0.5333333333333333 | 0.5925925 | 0.5333333
# resnet50 v3 f4  |  4     | 0.7                | 0.8125    | 0.7
# resnet50 v3 f4  |  5     | 0.4666666666666667 | 0.2413793 | 0.4666666
# resnet50 v3 f4  |  6     | 0.4333333333333333 | 0.3557692 | 0.4333333
#
# resnet50 v3 f5  |  1     | 0.5670731707317073 | 0.6804721 | 0.5670731
# resnet50 v3 f5  |  2     | 0.6036585365853658 | 0.7231113 | 0.6036585
# resnet50 v3 f5  |  3     | 0.5548780487804879 | 0.7023026 | 0.5548780
# resnet50 v3 f5  |  4     | 0.5365853658536586 | 0.7238398 | 0.5365853
# resnet50 v3 f5  |  5     | 0.5274390243902439 | 0.6052031 | 0.5274390
# resnet50 v3 f5  |  6     | 0.5121951219512195 | 0.5864978 | 0.5121951
#
# resnet50 v3 f6  |  1     | 0.4882352941176470 | 0.4777616 | 0.4882352
# resnet50 v3 f6  |  2     | 0.5019607843137255 | 0.7504911 | 0.5019607
# resnet50 v3 f6  |  3     | 0.5                | 0.5       | 0.5
# resnet50 v3 f6  |  4     | 0.5                | 0.5       | 0.5
# resnet50 v3 f6  |  5     | 0.4862745098039215 | 0.4757479 | 0.4862745
# resnet50 v3 f6  |  6     | 0.4470588235294120 | 0.4427443 | 0.4470588
#
# resnet50 v3 f7  |  1     | 0.6306818181818182 | 0.6308339 | 0.6306818
# resnet50 v3 f7  |  2     | 0.8295454545454546 | 0.8439353 | 0.8295455
# resnet50 v3 f7  |  3     | 0.8863636363636364 | 0.8895833 | 0.8863636
# resnet50 v3 f7  |  4     | 0.9204545454545454 | 0.9206718 | 0.9204545
# resnet50 v3 f7  |  5     | 0.8806818181818182 | 0.8847057 | 0.8806818
# resnet50 v3 f7  |  6     | 
#
# resnet50 v3 f8  |  1     | 0.4611872146118721 | 0.4400695 | 0.4611872
# resnet50 v3 f8  |  2     | 0.5730593607305936 | 0.7106782 | 0.5730594
# resnet50 v3 f8  |  3     | 0.5410958904109590 | 0.6610294 | 0.5410959
# resnet50 v3 f8  |  4     | 0.4634703196347032 | 0.4525666 | 0.4634703
# resnet50 v3 f8  |  5     | 0.8264840182648402 | 0.8569702 | 0.8264840
# resnet50 v3 f8  |  6     | 0.5547945205479452 | 0.5975646 | 0.5547945

<hr><br><br>

## `Colors_Down` classifier

<br><br>Load the train set, isolate the attribute as label, crop the image, and compute class weights

In [4]:
attribute  = 'colors_down' 

irm        = torchvision.io.ImageReadMode.RGB
split_val  = 640
dictionary = dict()
colors     = [
    'downblack', 'downwhite', 'downpink', 'downpurple', 'downyellow',
    'downgray', 'downblue', 'downgreen', 'downbrown'
]

for image_file in tqdm(images):
    
    image_id  = int(image_file.split('/')[-1].split('_')[0])
    array     = [(annotations.loc[image_id][color] - 1) for color in colors]
    tensor    = torchvision.io.read_image( image_file,irm ).float()
    
    if 1 in array:
        label = (array.index(1) + 1)
    else:
        label = 0
    
    ct = list()
    for c,channel in enumerate(tensor):
        ct.append( tensor[c][60:120].numpy() )
    
    tensor    = torch.tensor(ct).float()
    
    channels, height, width = tensor.shape
    tensor    = tensor.reshape(1, channels, height, width)
    
    if image_id not in dictionary:
        dictionary[image_id] = list()
        dictionary[image_id].append([tensor, label])
    else:
        dictionary[image_id].append([tensor, label])
    
data = list(dictionary.items())
random.Random(3).shuffle(data)

train_set, test_set = project.split_dataset(data,split_val) 

  0%|          | 0/12989 [00:00<?, ?it/s]

<hr><br><br>Fit a pretrained ResNet50 architecture

In [ ]:
v = '3_2'                       # model version 
e =  0                          # starting epoch
f =  0                          # feature index 


model     = torchvision.models.resnet50(pretrained=True) 
model.fc  = nn.Linear(
    in_features=model.fc.in_features,
    out_features=2, bias=True
)

if e > 0:
    w = './models/{}_resnet50.v{}.x{}.net'.format(attribute, v, e)
    model.load_state_dict( torch.load(w) )


optimizer = optim.AdamW(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


for i in range(10): 

    model_name = '{}_resnet50.v{}.x{}.net'.format(attribute, v, e+i+1)

    p = project.train(model, train_set,
        criterion, optimizer, epochs=1, li=500, monobatch=True
    )

    r = project.test(model, test_set, monobatch=True)

    project.save(model, './models/{}'.format(model_name))

In [ ]:
# SCORE BOARD 
# 
# net_design_name    | epochs | accuracy           | precision | recall
#                    |        |                    |           |
# resnet50 v3.2 f0   |  1     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f0   |  2     | 0.505              | 0.5512821 | 0.505
# resnet50 v3.2 f0   |  3     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f0   |  4     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f0   |  5     | 0.65               | 0.6860119 | 0.65
# resnet50 v3.2 f0   |  6     | 0.46               | 0.4523810 | 0.46
# resnet50 v3.2 f0   |  7     | 0.48               | 0.43      | 0.48
# resnet50 v3.2 f0   |  8     | 0.505              | 0.7512563 | 0.505
# resnet50 v3.2 f0   |  9     | 0.49               | 0.2474747 | 0.49
# resnet50 v3.2 f0   | 10     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f0   | 11     | 0.495              | 0.4487179 | 0.495
# resnet50 v3.2 f0   | 12     | 0.515              | 0.5160000 | 0.515
# resnet50 v3.2 f0   | 13     | 0.47               | 0.4377076 | 0.47
# resnet50 v3.2 f0   | 14     | 0.48               | 0.4584718 | 0.48
# resnet50 v3.2 f0   | 15     | 0.34               | 0.3283998 | 0.34
# resnet50 v3.2 f0   | 16     | 0.215              | 0.1888307 | 0.215
# resnet50 v3.2 f0   | 17     | 0.345              | 0.3391949 | 0.345
# resnet50 v3.2 f0   | 18     | 0.325              | 0.3184459 | 0.325
# resnet50 v3.2 f0   | 19     | 0.435              | 0.3820541 | 0.435
# resnet50 v3.2 f0   | 20     | 0.44               | 0.4125874 | 0.44
#                    |        |                    |           |
# resnet50 v3.2 f1   |  1     | 0.6005625879043600 | 0.6012839 | 0.6005626
# resnet50 v3.2 f1   |  2     | 0.7285513361462729 | 0.7352526 | 0.7285513
# resnet50 v3.2 f1   |  3     | 0.7623066104078763 | 0.7694796 | 0.7623066
# resnet50 v3.2 f1   |  4     | 0.6863572433192686 | 0.7515709 | 0.6863572
# resnet50 v3.2 f1   |  5     | 0.7475386779184248 | 0.7910844 | 0.7475387
# resnet50 v3.2 f1   |  6     | 0.8234880450070323 | 0.8367309 | 0.8234880
#                    |        |                    |           |
# resnet50 v3.2 f2   |  1     | 0.45               | 0.4446259 | 0.4504505
# resnet50 v3.2 f2   |  2     | 0.518              | 0.5228395 | 0.5180180
# resnet50 v3.2 f2   |  3     | 0.765              | 0.7671535 | 0.7657658
# resnet50 v3.2 f2   |  4     | 0.59               | 0.6027778 | 0.5900901
# resnet50 v3.2 f2   |  5     | 0.738              | 0.7485005 | 0.7387387
# resnet50 v3.2 f2   |  6     | 0.522              | 0.5245640 | 0.5225225
# resnet50 v3.2 f2   |  7     | 0.797              | 0.8131840 | 0.7972973
# resnet50 v3.2 f2   |  8     | 0.509              | 0.5856481 | 0.5090090
# resnet50 v3.2 f2   |  9     | 0.838              | 0.8388278 | 0.8378378
# resnet50 v3.2 f2   | 10     | 0.789              | 0.8391902 | 0.7882883
# resnet50 v3.2 f2   | 11     | 0.806              | 0.8200475 | 0.8063063
# resnet50 v3.2 f2   | 12     | 0.788              | 0.7989899 | 0.7882883
# resnet50 v3.2 f2   | 13     | 0.608              | 0.6107048 | 0.6081081
# resnet50 v3.2 f2   | 14     | 0.662              | 0.6660572 | 0.6621622
# resnet50 v3.2 f2   | 15     | 0.477              | 0.4393708 | 0.4774775
#                    |        |                    |           |
# resnet50 v3.2 f3   |  1     | 0.6730769230769231 | 0.6866029 | 0.6730769
# resnet50 v3.2 f3   |  2     | 0.7692307692307693 | 0.8421053 | 0.7692308
# resnet50 v3.2 f3   |  3     | 0.5769230769230769 | 0.5849673 | 0.5769231
# resnet50 v3.2 f3   |  4     | 0.5384615384615384 | 0.5738636 | 0.5384615
# resnet50 v3.2 f3   |  5     | 0.25               | 0.2403994 | 0.2500000
# resnet50 v3.2 f3   |  6     | 0.1923076923076932 | 0.1848485 | 0.1923077
# resnet50 v3.2 f3   |  7     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f3   |  8     | 0.5576923076923077 | 0.6659574 | 0.5576923
# resnet50 v3.2 f3   |  9     | 0.9038461538461539 | 0.9193548 | 0.9038462
# resnet50 v3.2 f3   | 10     | 0.9615384615384616 | 0.9615385 | 0.9615385
# resnet50 v3.2 f3   | 11     | 0.9038461538461543 | 0.9044444 | 0.9038462
# resnet50 v3.2 f3   | 12     | 0.5961538461538461 | 0.7765957 | 0.5961538
# resnet50 v3.2 f3   | 13     | 0.9230769230769231 | 0.9255952 | 0.9230769
# resnet50 v3.2 f3   | 14     | 0.9230769230769231 | 0.9230769 | 0.9230769
# resnet50 v3.2 f3   | 15     | 0.9615384615384616 | 0.9642857 | 0.9615385
#                    |        |                    |           |
# resnet50 v3.2 f4   |  1     | 0                  | 0         | 0
# resnet50 v3.2 f4   |  2     | 0.9974632166412988 | 0.5       | 0.4987316
# resnet50 v3.2 f4   |  3     | 0.9994926433282597 | 0.5       | 0.4997463
# resnet50 v3.2 f4   |  4     | 1                  | 1         | 1
# resnet50 v3.2 f4   |  5     | 0.9984779299847792 | 0.5       | 0.4992390
# resnet50 v3.2 f4   |  6     | 0.9994926433282597 | 0.5       | 0.4997463
# resnet50 v3.2 f4   |  7     | 0                  | 0         | 0
# resnet50 v3.2 f4   |  8     | 0.5606291222729579 | 0.5       | 0.2803146
# resnet50 v3.2 f4   |  9     | 0.9502790461694571 | 0.5       | 0.4751395
# resnet50 v3.2 f4   | 10     | 1                  | 1         | 1
# resnet50 v3.2 f4   | 11     | 0.8807711821410451 | 0.5       | 0.4403856
# resnet50 v3.2 f4   | 12     | 0.9994926433282697 | 0.5       | 0.4997463
#                    |        |                    |           |
# resnet50 v3.2 f5   |  1     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f5   |  2     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f5   |  3     | 0.4673913043478261 | 0.4402597 | 0.4673913
# resnet50 v3.2 f5   |  4     | 0.3695652173913043 | 0.3600406 | 0.3695652
# resnet50 v3.2 f5   |  5     | 0.5543478260869565 | 0.5686567 | 0.5543478
# resnet50 v3.2 f5   |  6     | 0.7282608695652174 | 0.8043478 | 0.7282609
# resnet50 v3.2 f5   |  7     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f5   |  8     | 0.4782608695652174 | 0.4654655 | 0.4782609
# resnet50 v3.2 f5   |  9     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f5   | 10     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f5   | 11     | 0.75               | 0.7651629 | 0.75
# resnet50 v3.2 f5   | 12     | 0.5978260869565217 | 0.7771084 | 0.5978261
# resnet50 v3.2 f5   | 13     | 0.7065217391304348 | 0.7066194 | 0.7065217
# resnet50 v3.2 f5   | 14     | 0.5108695652173914 | 0.7527473 | 0.5108696
# resnet50 v3.2 f5   | 15     | 0.7391304347826086 | 0.7720430 | 0.7391304
#                    |        |                    |           |
# resnet50 v3.2 f6   |  1     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f6   |  2     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f6   |  3     | 0.5013             | 0.5150285 | 0.5012626
# resnet50 v3.2 f6   |  4     | 0.5051             | 0.5429688 | 0.5050505
# resnet50 v3.2 f6   |  5     | 0.5682             | 0.5685244 | 0.5681818
# resnet50 v3.2 f6   |  6     | 0.6149             | 0.6148997 | 0.6148990
# resnet50 v3.2 f6   |  7     | 0.5619             | 0.5620781 | 0.5618687
# resnet50 v3.2 f6   |  8     | 0.5947             | 0.6534234 | 0.5946970
# resnet50 v3.2 f6   |  9     | 0.639              | 0.6406250 | 0.639
# resnet50 v3.2 f6   | 10     | 0.6098             | 0.6233468 | 0.6098485
# resnet50 v3.2 f6   | 11     | 0.6313             | 0.6366133 | 0.6313131
# resnet50 v3.2 f6   | 12     | 0.6022             | 0.6058705 | 0.6022727
#                    |        |                    |           |
# resnet50 v3.2 f7   |  1     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f7   |  2     | 0.4988207547169811 | 0.4983106 | 0.4988208
# resnet50 v3.2 f7   |  3     | 0.5625             | 0.5779980 | 0.5625
# resnet50 v3.2 f7   |  4     | 0.5377358490566038 | 0.5604764 | 0.5377358
# resnet50 v3.2 f7   |  5     | 0.5047169811320755 | 0.5466652 | 0.5047170
# resnet50 v3.2 f7   |  6     | 0.5990566037735849 | 0.6320755 | 0.5990566
# resnet50 v3.2 f7   |  7     | 0.7169811320754716 | 0.7169860 | 0.7169811
# resnet50 v3.2 f7   |  8     | 0.6933962264150944 | 0.7052760 | 0.6933962
# resnet50 v3.2 f7   |  9     | 0.6969339622641509 | 0.7000599 | 0.6969340
# resnet50 v3.2 f7   | 10     | 0.6603773584905660 | 0.6719507 | 0.6603774
# resnet50 v3.2 f7   | 11     | 0.7158018867924528 | 0.7335884 | 0.7158019
# resnet50 v3.2 f7   | 12     | 0.7040094339622641 | 0.7105988 | 0.7040094
#                    |        |                    |           |
# resnet50 v3.2 f8   |  1     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f8   |  2     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f8   |  3     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f8   |  4     | 0.5119047619047619 | 0.5227519 | 0.5119048
# resnet50 v3.2 f8   |  5     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f8   |  6     | 0.4880952380952381 | 0.4468354 | 0.4880952
# resnet50 v3.2 f8   |  7     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f8   |  8     | 0.3809523809523809 | 0.3069853 | 0.3809524
# resnet50 v3.2 f8   |  9     | 0.4880952380952381 | 0.2469880 | 0.4880952
# resnet50 v3.2 f8   | 10     | 0.4880952380952318 | 0.4841270 | 0.4880952
# resnet50 v3.2 f8   | 11     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f8   | 12     | 0.4523809523809524 | 0.4513889 | 0.4523810
# resnet50 v3.2 f8   | 13     | 0.4404761904761904 | 0.3862405 | 0.4404762
# resnet50 v3.2 f8   | 14     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f8   | 15     | 0.5                | 0.25      | 0.5
#                    |        |                    |           |
# resnet50 v3.2 f9   |  1     | 0.473913043478261  | 0.4390028 | 0.4739130
# resnet50 v3.2 f9   |  2     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f9   |  3     | 0.717391304347826  | 0.7178030 | 0.7173913
# resnet50 v3.2 f9   |  4     | 0.473913043478261  | 0.4721415 | 0.4739130
# resnet50 v3.2 f9   |  5     | 0.691304347826087  | 0.6913188 | 0.6913043
# resnet50 v3.2 f9   |  6     | 0.756521739130435  | 0.7565993 | 0.7565217
# resnet50 v3.2 f9   |  7     | 0.726086956521739  | 0.7311379 | 0.7260870
# resnet50 v3.2 f9   |  8     | 0.760869565217391  | 0.7610472 | 0.7608696
# resnet50 v3.2 f9   |  9     | 0.552173913043478  | 0.5533395 | 0.5521739
# resnet50 v3.2 f9   | 10     | 0.673913043478261  | 0.7053571 | 0.6739130
# resnet50 v3.2 f9   | 11     | 0.678260869565217  | 0.6976276 | 0.6782609
# resnet50 v3.2 f9   | 12     | 0.673913043478261  | 0.6769231 | 0.6739130

<br><br>